In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [16]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS-nanodegree/Project 1/data/clean_data/data_cleaned.csv', index_col=0)

In [17]:
df.head()

,id,host_id,host_since,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,city,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,...,Houseboat,Hut,Island,Lighthouse,Loft,Minsu (Taiwan),Nature lodge,Other.1,Plane,Ryokan (Japan),Serviced apartment,"Shepherd's hut (U.K., France)",Tent,Tiny house,Townhouse,Treehouse,Villa,Yurt,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,Entire home/apt,Hotel room,Private room,Shared room,flexible,luxury_moderate,luxury_super_strict_95,moderate,strict,strict_14_with_grace_period,super_strict_30,super_strict_60,0,a few days or more,within a day,within a few hours,within an hour
0,11551,43039,4517.0,100,1,3.0,1,1,london,51.46225,-0.11732,4,1.0,1.0,1.0,4.477337,5.991465,3.401197,2,2.995732,2,180,2,2,180,180,2.0,180.0,1,184,1,92.0,9.0,9.0,10.0,10.0,9.0,9.0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,13913,54730,4473.0,100,0,4.0,1,0,islington,51.56802,-0.11121,2,1.0,1.0,0.0,4.174387,4.605170,2.708050,1,2.708050,1,29,1,1,29,29,1.0,29.0,1,16,2,96.0,9.0,10.0,9.0,10.0,9.0,9.0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,15400,60302,4454.0,100,1,1.0,1,1,london,51.48796,-0.16898,2,1.0,1.0,1.0,4.605170,5.010635,3.912023,2,-55.000000,30,50,30,30,50,50,30.0,50.0,1,87,6,96.0,10.0,10.0,10.0,10.0,10.0,9.0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,17402,67564,4424.0,98,1,18.0,1,1,london,51.52098,-0.14002,6,2.0,3.0,4.0,5.703782,5.857933,4.174387,4,2.302585,3,365,2,3,365,365,3.0,365.0,1,41,5,94.0,10.0,9.0,9.0,9.0,10.0,9.0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,24328,41759,4522.0,80,0,0.0,1,1,london,51.47298,-0.16376,2,1.5,1.0,1.0,5.164786,5.521461,4.248495,2,-55.000000,30,1125,30,30,1125,1125,30.0,1125.0,1,93,1,98.0,10.0,10.0,10.0,10.0,9.0,9.0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


## Feature Selection

In [18]:
df = pd.DataFrame.drop(df, columns=['id', 'host_id'])

In [19]:
df['price'].describe()

count    84734.000000
mean         4.419075
std          1.173961
min        -55.000000
25%          3.806662
50%          4.418841
75%          4.941642
max          9.903488
Name: price, dtype: float64

In [5]:
df = df.replace([np.inf, -np.inf], np.nan).dropna()

In [20]:
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(df.select_dtypes(include=numerics).columns)
df = df[numerical_vars]
df.shape

(84734, 333)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df, df['price'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((59313, 333), (25421, 333))

In [22]:
df.head()

,host_since,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count_entire_homes,...,Houseboat,Hut,Island,Lighthouse,Loft,Minsu (Taiwan),Nature lodge,Other.1,Plane,Ryokan (Japan),Serviced apartment,"Shepherd's hut (U.K., France)",Tent,Tiny house,Townhouse,Treehouse,Villa,Yurt,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,Entire home/apt,Hotel room,Private room,Shared room,flexible,luxury_moderate,luxury_super_strict_95,moderate,strict,strict_14_with_grace_period,super_strict_30,super_strict_60,0,a few days or more,within a day,within a few hours,within an hour
0,4517.0,100,1,3.0,1,1,51.46225,-0.11732,4,1.0,1.0,1.0,4.477337,5.991465,3.401197,2,2.995732,2,180,2,2,180,180,2.0,180.0,1,184,1,92.0,9.0,9.0,10.0,10.0,9.0,9.0,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,4473.0,100,0,4.0,1,0,51.56802,-0.11121,2,1.0,1.0,0.0,4.174387,4.605170,2.708050,1,2.708050,1,29,1,1,29,29,1.0,29.0,1,16,2,96.0,9.0,10.0,9.0,10.0,9.0,9.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,4454.0,100,1,1.0,1,1,51.48796,-0.16898,2,1.0,1.0,1.0,4.605170,5.010635,3.912023,2,-55.000000,30,50,30,30,50,50,30.0,50.0,1,87,6,96.0,10.0,10.0,10.0,10.0,10.0,9.0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,4424.0,98,1,18.0,1,1,51.52098,-0.14002,6,2.0,3.0,4.0,5.703782,5.857933,4.174387,4,2.302585,3,365,2,3,365,365,3.0,365.0,1,41,5,94.0,10.0,9.0,9.0,9.0,10.0,9.0,1,0,0,0,15,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,4522.0,80,0,0.0,1,1,51.47298,-0.16376,2,1.5,1.0,1.0,5.164786,5.521461,4.248495,2,-55.000000,30,1125,30,30,1125,1125,30.0,1125.0,1,93,1,98.0,10.0,10.0,10.0,10.0,9.0,9.0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [23]:
scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

StandardScaler()

In [24]:
sel_ = Lasso(alpha=0.0000001, max_iter=1e3)
sel_.fit(X_train, y_train)
print('training set:', sel_.score(X_train, y_train))
print('validation set:', sel_.score(X_test, y_test))
# y_pred_train = sel_.predict(X_train)
# y_pred_val = sel_.predict(X_test)

print('total number of parameters:', sum(sel_.coef_!=0))

training set: 0.9999999291291853
validation set: 0.9999999334284823
total number of parameters: 169


In [25]:
coefs = np.array(sel_.coef_!=0)
np.save('/content/drive/MyDrive/Colab Notebooks/DS-nanodegree/Project 1/data/clean_data/selected_coefs.npy', coefs)

## Export data with selected columns

In [26]:
def feature_selected(df, coeffs):
    df = pd.DataFrame.drop(df, columns=['id', 'host_id'])
    
    numerics = ['int16','int32','int64','float16','float32','float64']
    numerical_vars = list(df.select_dtypes(include=numerics).columns)
    df = df[numerical_vars]

    col_set = []

    for i in range(len(coeffs)):
        if (coeffs[i]):
            col_set.append(df.columns[i])

    df = df[list(col_set)]

    return df

In [27]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS-nanodegree/Project 1/data/clean_data/data_cleaned.csv', index_col=0)
coeffs = np.load('/content/drive/MyDrive/Colab Notebooks/DS-nanodegree/Project 1/data/clean_data/selected_coefs.npy')

In [28]:
df_selected = feature_selected(df, coeffs)
df_selected.to_csv('/content/drive/MyDrive/Colab Notebooks/DS-nanodegree/Project 1/data/clean_data/data_selected.csv')

In [29]:
df_selected.shape

(84734, 169)